## Scarping web site project
v.2  
Autor: Alex Feldman  
email: amgoodness@gmail.com

Project goals:
- Collect data about Israel's doctors from www.medonline.co.il
- Extract data from the site, make dataset, transform to data required format.
- Save data to csv files (transformed and raw data).

We have a web site with information about doctors. Each doctor has a personal page with data. Links for personal pages are on the 'doctors-search' page. One 'doctors-search' page includes 30 links. There are 34 'doctors-search' pages.
Links to personal pages have duplicates. Need drop ones.

#### Working plan:
1. Create a list of all links to personal pages.
2. Extract data from personal pages and save it to 2 datasets (because there are 2 different formats of data)
3. Transform data to required columns by regular exprassions (columns telephone, degree, city)
4. Convert the result to 2 Excel files (transformed columns and transformed + raw columns).

### Table of Contents

1. [Import libraries.](#1)  
2. [Connect to web pages.](#2)
3. [Collecting links from the web site.](#3)
4. [Extract data from personal pages.](#4)
5. [Transform and save data.](#5)
6. [Conclusion.](#6)

## 1. Import libraries<a class="anchor" id="1"></a>

In [ ]:
import json
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import numpy as np
import re



## 2. Connect to web pages <a class="anchor" id="2"></a>

Make the function for creating a driver (selenium object) for connecting to pages.

In [ ]:

URL = 'https://www.medonline.co.il/doctors-search'
def connect(url=URL):
    """
    Connect to web page
    :param url: link to web page
    :return: chrome driver object
    """
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    driver = webdriver.Chrome(options=options)
    try:
        driver.get(url)
        time.sleep(1)
    except TimeoutException:
        print('new connection try')
        try:
            driver.get(url)
            time.sleep(5)
        except:
            print('Can\'t open web page:', url)
            
    return driver

## 3. Collecting links from the web site <a class="anchor" id="3"></a>
Create a page listing engine. Make functions to extract links from pages and save them to text files.

In [45]:
def get_links_from_one_page(url_page):
    """
    To collect links one page
    :param my_webpage: link to page for scrapping: selenium obj
    :return: links from one page: person_urls[str]
    """
    links_page = []
    driver = connect(url_page)
    tab = driver.find_element_by_xpath('//*[@id="right_block_s_d"]')
    list_items = tab.find_elements_by_class_name('search_result')
    for i in range(len(list_items)):
        item = list_items[i].find_element_by_tag_name('a')
        links_page.append(item.get_attribute('href'))
    driver.close()
    return links_page 

def get_all_links(num_pages=34):
    """
    Collect links from all site by list pages
    :param num_pages: num pages on site
    :return: links list to personal page for site
    """
    all_pages_links = []
    for page in range(1, num_pages+1):
        url_page = URL+'?page='+str(page)
        links_page = get_links_from_one_page(url_page)
        all_pages_links.append(links_page)
    return all_pages_links

In [34]:
# get links from site

links_list = get_all_links()
path_pattern = 'www.medonline.co.il/expert/'
# write down links to the txt file
check_dup_list=[]
output_links = open('links.txt', 'w')
wrong_links = open('wrong_links.txt', 'w')
for pg in links_list:
    for url_str in pg:
        check_dup_list.append(url_str)
# check duplicate url
check_dup_list=list(dict.fromkeys(check_dup_list))
for l in check_dup_list:
    if l.__contains__(path_pattern):
        output_links.write(l + '\n')
    else:
        wrong_links.write(l + '\n')
        
output_links.close()
wrong_links.close()

print(f'links collected in the file successfully')


links collected in the file successfully


## 4. Extract data from personal pages <a class="anchor" id="4"></a>
Read a text file with personal page links. Extract data from page end save it to dataframes and csv files. 

In [42]:
def extract_data_type_1(driver):
    personal_dict={}
    # found frame with data
    tab_doc = driver.find_element_by_class_name('profile_doc')

    # found elements in right table
    doc_info_subtab = tab_doc.find_element_by_id('doc_information')
    doc_info_li = doc_info_subtab.find_elements_by_tag_name('li')
    items_doc_info = {}
    for li in range(len(doc_info_li)):
        items_doc_info[li] = doc_info_li[li].find_element_by_tag_name('b').text

    # found elements in right table
    doc_info__left_subtab = tab_doc.find_element_by_id('doc_information_left')
    doc_info__left_li = doc_info__left_subtab.find_elements_by_tag_name('li')
    items_doc_info_left = {}
    for li in range(len(doc_info__left_li)):
        items_doc_info_left[li] = doc_info__left_li[li].find_element_by_tag_name('b').text
    doc_about = driver.find_element_by_id('short_text')
    item_description = doc_about.find_element_by_tag_name('p').text
    driver.close()
    # make dict element
    personal_dict = {'name': items_doc_info[0],
                               'degree': items_doc_info[1],
                               'specialization': items_doc_info[2],
                               'care_of': items_doc_info[3],
                               'type': items_doc_info[4],
                               'area': items_doc_info[5],
                               'activity_time': items_doc_info_left[0],
                               'address': items_doc_info_left[1],
                               'telephon': items_doc_info_left[2],
                               'email': items_doc_info_left[3],
                               'manage_forums': items_doc_info_left[4],
                               'description': item_description
                               }
    return personal_dict


def extract_data_type_2(driver):
    personal_dict2 = {}
    tab_doc = driver.find_element_by_class_name('search_result')
    item_title = tab_doc.find_element_by_class_name('title').text
    item_specialization = tab_doc.find_element_by_class_name('smalltitle').text
    item_description = tab_doc.find_element_by_tag_name('p').text
    item_contact_info = tab_doc.find_element_by_class_name('other').text
    driver.close()
    # make dict element
    personal_dict2 = {
        'title': item_title,
        'specialization': item_specialization,
        'description': item_description,
        'contact_info': item_contact_info,
    }
    return personal_dict2


In [57]:
# get data from personal pages
items_dict = {}
items_dict_2 = {}
with open('links.txt', 'r') as f:
    link = f.readlines()
    for i in np.arange(0, len(link)):
        personal_url = link[i]
        try:
            driver=connect(personal_url)
        except WebDriverException:
            print('WebDriverException say Hallo')
            continue
        # we have 2 different variants of the table with data
        try:  # if variant the table #1
            items_dict[personal_url] = extract_data_type_1(driver)
            
        except:  # if variant the table #2
            try:
                items_dict_2[personal_url] = extract_data_type_2(driver)
                
            except:
                print('Can\'t extract data from url {}'.format(personal_url))
                pass
        
            # create dataset
docs_db = pd.DataFrame.from_dict(items_dict, orient='index',
                                columns=['name', 'degree', 'specialization', 'care_of', 'type', 'area',
                                         'activity_time', 'address', 'telephon', 'email', 'manage_forums',
                                         'description']).reset_index()

docs_db_2 = pd.DataFrame.from_dict(items_dict_2, orient='index',
                                       columns=['title', 'specialization', 'description', 'contact_info']).reset_index()
print('Datasets created')
# save to csv
docs_db.to_csv('docs_db.csv', index=False)
docs_db_2.to_csv('docs_db_2.csv', index=False)
print('CSV files wrote to the disk')

Can't extract data from url https://www.medonline.co.il/expert/%D7%A8%D7%91%D7%99%D7%99-%D7%9E%D7%A8%D7%98%D7%99%D7%9F

Can't extract data from url https://www.medonline.co.il/expert/%D7%90%D7%93%D7%99-%D7%90%D7%9C%D7%91%D7%99%D7%9C%D7%99%D7%94-%D7%9E%D7%93%D7%A8%D7%A1%D7%99%D7%9D-%D7%91%D7%9E%D7%A8%D7%9B%D7%96-%D7%91%D7%A9%D7%A8%D7%95%D7%9F

Can't extract data from url https://www.medonline.co.il/expert/%D7%A4%D7%A8%D7%95%D7%A4-%D7%92%D7%99%D7%95%D7%A8%D7%90-%D7%A4%D7%99%D7%9C%D7%A8-%D7%99%D7%9C%D7%93%D7%99%D7%9D-%D7%9B%D7%A8%D7%9E%D7%9C

Can't extract data from url https://www.medonline.co.il/expert/%D7%99%D7%90%D7%99%D7%A8-%D7%94%D7%95%D7%93

Datasets created
CSV files wrote on disk


## 5. Transform and save data <a class="anchor" id="5"></a>
Transform data by regex and save it.

In [568]:
# open data from drive if it's need
#docs_db_2 = pd.read_csv('docs_db_2.csv')
#docs_db = pd.read_csv('docs_db.csv')

In [60]:

# make the degree column from the title
docs_db_2['degree'] = docs_db_2['title'].apply(lambda x: 'דוקטור' if re.findall('דוקטור|ד"ר',x) 
                                               else ("פרופסור" if re.findall("פרופ'|פרופסור",x)
                                               else ''))

# extract telephon from contact info
docs_db_2['telephon'] = docs_db_2['contact_info'].apply(lambda x: re.findall(r'[0]\d+-\d{7}|\d{8,10}',x))
# clean data
docs_db_2['telephon'] = docs_db_2['telephon'].apply(lambda x: re.sub(r'[\[\,\]]','',str(x)))
docs_db_2['telephon'] = docs_db_2['telephon'].apply(lambda x: re.sub(r'[\"\']','',str(x)))

In [61]:
# set list of the israel cities 
# (from https://he.wikipedia.org/wiki/%D7%A2%D7%A8%D7%99%D7%9D_%D7%91%D7%99%D7%A9%D7%A8%D7%90%D7%9C)
list_city = ["ירושלים","תל אביב-יפו","חיפה","ראשון לציון","פתח תקווה","אשדוד","נתניה","באר שבע","בני ברק","חולון","רמת גן","אשקלון","רחובות","בת ים","בית שמש","כפר סבא","הרצליה","חדרה","מודיעין- מכבים- רעות","נצרת","לוד","רמלה","רעננה","מודיעין עילית","רהט","הוד השרון","גבעתיים","קריית אתא","נהריה","ביתר עילית","אום אל-פחם","קריית גת","אילת","ראש העין","עפולה","נס ציונה","עכו","אלעד","רמת השרון","כרמיאל","יבנה","טבריה","טייבה","קריית מוצקין","שפרעם","נוף הגליל","קריית ים","קריית ביאליק","קריית אונו","מעלה אדומים","אור יהודה","צפת","נתיבות","דימונה","טמרה","סח'נין","יהוד-מונוסון","באקה אל-גרבייה","אופקים","גבעת שמואל","טירה","ערד","מגדל העמק","שדרות","עראבה","נשר","קריית שמונה","יקנעם עילית","כפר קאסם","כפר יונה","קלנסווה","קריית מלאכי" ,"תל אב'","תל אב","מעלות- תרשיחא","טירת כרמל","אריאל","אור עקיבא","בית שאן"
]


In [63]:
# extract city from the contact info by list of city

# make pattern for regex
ex=''
for i in list_city:
    if i==0:
        ex = ex+i
    else:
        ex = ex+'|'+i
# extract and clean data
docs_db_2['city'] = docs_db_2['contact_info'].apply(lambda x: re.findall(ex,x))
docs_db_2['city'] = docs_db_2['city'].apply(lambda x: re.findall(r'\w+',str(x)))
docs_db_2['city'] = docs_db_2['city'].apply(lambda x: re.sub(r'[\[\,\]]','',str(x)))
docs_db_2['city'] = docs_db_2['city'].apply(lambda x: re.sub(r'[\"\']','',str(x)))

In [65]:
# for docs_db extract area city from the address by list of city

# make pattern for regex
ex=''
for i in list_city:
    if i==0:
        ex = ex+i
    else:
        ex = ex+'|'+i
# extract and clean data
docs_db['city'] = docs_db['address'].apply(lambda x: re.findall(ex,x))
docs_db['city'] = docs_db['city'].apply(lambda x: re.findall(r'\w+',str(x)))
docs_db['city'] = docs_db['city'].apply(lambda x: re.sub(r'[\[\,\]]','',str(x)))
docs_db['city'] = docs_db['city'].apply(lambda x: re.sub(r'[\"\']','',str(x)))

docs_db['contact_info'] = docs_db.apply(lambda x: str(x['address'])+'. '+str(x['email'])+'. '
                                        +str(x['activity_time']), axis=1)
docs_db['description'] = docs_db.apply(lambda x: str(x['description'])+'. Care of: '+str(x['care_of'])+'. Manage forums: '
                                        +str(x['manage_forums']), axis=1)


In [66]:
docs_db_2 = docs_db_2.rename(columns={'title':'name'})

doc_db_full = pd.concat([docs_db, docs_db_2], ignore_index=True)

final_doc_db = doc_db_full[['index', 'name', 'degree', 'specialization', 'telephon', 'city', 'description','contact_info']]

doc_db_full.to_excel('doc_db_full.xlsx')
final_doc_db.to_excel('final_doc_db.xlsx')

## 6. Conclusion <a class="anchor" id="6"></a>
As a result, we got 2 excel files. The first one with required columns and the second one with raw and transformed columns.

In [68]:
final_doc_db.head()

,index,name,degree,specialization,telephon,city,description,contact_info
0,https://www.medonline.co.il/expert/%D7%99%D7%A...,"ד""ר יעקב גלמן",דוקטור,טיפולי הומיאופתיה,077-2317132,שדרות חיפה,"ד""ר יעקב גלמן הינו רופא קונבנציונאלי ותיק ומנו...","שדרות ההסתדרות 55, חיפה. yakov0511@gmail.com..."
1,https://www.medonline.co.il/expert/%D7%90%D7%A...,"רופא שיניים בראשון לציון - ד""ר אריאל סביון",דוקטור לרפואת שיניים,"רפואת שיניים מיקרוסקופית, טיפול במחלות חניכיים...",077-9975999,ראשון לציון,"ד""ר אריאל סביון הינו המייסד והבעלים של ""סביון ...","רחוב ז'בוטינסקי 7, ראשון לציון. dr.savion@gm..."
2,https://www.medonline.co.il/expert/%D7%9E%D7%A...,"ד""ר משה שמש",דוקטור,"אסתטיקה, אסתטיקת הפנים, אסתטיקה רפואית, אנטי א...",077-2318148,תל אב,"ד""ר משה שמש עוסק ברפואה אסתטית בדגש על טיפולים...","רחוב הברזל 11, קומה 3 B-16, רמת החייל, תל אב..."
3,https://www.medonline.co.il/expert/%D7%90%D7%9...,"ד""ר אורן בן לולו",דוקטור,"אורתופד מומחה, החלפת מפרק ירך, החלפת מפרק ברך",04-6069906,שדרות חיפה,"ד""ר אורן בן לולו הינו רופא אורתופד מומחה, המתמ...","שדרות ההסתדרות 55, קניון לב המפרץ, אסותא, חי..."
4,https://www.medonline.co.il/expert/%D7%A7%D7%A...,"ד""ר ופא קסיס",דוקטור,"גריאטריה, פנימית, רפואת כאב, פסיכוגריאטריה",077-2319-148,חיפה,"ד""ר ופא קסיס הינה מומחית לרפואה פנימית וגריאטר...","נווה גנים 12, חיפה. wafakasis.dr@gmail.com. ..."


In [69]:
doc_db_full.head()

,index,name,degree,specialization,care_of,type,area,activity_time,address,telephon,email,manage_forums,description,city,contact_info
0,https://www.medonline.co.il/expert/%D7%99%D7%A...,"ד""ר יעקב גלמן",דוקטור,טיפולי הומיאופתיה,"ילדים, מבוגרים, גיל הזהב, נוער",רפואה קונבנציונלית,צפון,"שר""פ+ קניון לב המפרץ, קומה 3.\nזימון תורים: 07...","שדרות ההסתדרות 55, חיפה",077-2317132,yakov0511@gmail.com,לא קיים פורום המנוהל ע''י מומחה זה.,"ד""ר יעקב גלמן הינו רופא קונבנציונאלי ותיק ומנו...",שדרות חיפה,"שדרות ההסתדרות 55, חיפה. yakov0511@gmail.com..."
1,https://www.medonline.co.il/expert/%D7%90%D7%A...,"רופא שיניים בראשון לציון - ד""ר אריאל סביון",דוקטור לרפואת שיניים,"רפואת שיניים מיקרוסקופית, טיפול במחלות חניכיים...","ילדים, מבוגרים, גיל הזהב, נוער",רפואה קונבנציונלית,מרכז,יום ראשון - שני: 09:00 - 20:00\nיום שלישי 14:0...,"רחוב ז'בוטינסקי 7, ראשון לציון",077-9975999,dr.savion@gmail.com,פורום טיפול במחלות חניכיים בלייזר,"ד""ר אריאל סביון הינו המייסד והבעלים של ""סביון ...",ראשון לציון,"רחוב ז'בוטינסקי 7, ראשון לציון. dr.savion@gm..."
2,https://www.medonline.co.il/expert/%D7%9E%D7%A...,"ד""ר משה שמש",דוקטור,"אסתטיקה, אסתטיקת הפנים, אסתטיקה רפואית, אנטי א...","מבוגרים, גיל הזהב, נוער",רפואה קונבנציונלית,מרכז,ימי א'-ה' מ- 10:00 עד 19:00\nיום שישי מ- 10:00...,"רחוב הברזל 11, קומה 3 B-16, רמת החייל, תל אביב",077-2318148,mosesshemesh@gmail.com,פורום מתיחת פנים ללא ניתוח,"ד""ר משה שמש עוסק ברפואה אסתטית בדגש על טיפולים...",תל אב,"רחוב הברזל 11, קומה 3 B-16, רמת החייל, תל אב..."
3,https://www.medonline.co.il/expert/%D7%90%D7%9...,"ד""ר אורן בן לולו",דוקטור,"אורתופד מומחה, החלפת מפרק ירך, החלפת מפרק ברך","מבוגרים, גיל הזהב, נוער",רפואה קונבנציונלית,צפון,ימי שני ורביעי בשעות: 16:00 עד 20:30\nכתובת למ...,"שדרות ההסתדרות 55, קניון לב המפרץ, אסותא, חיפה",04-6069906,benluluoren@gmail.com,"פורום החלפת מפרק ירך, פורום החלפת מפרק ברך","ד""ר אורן בן לולו הינו רופא אורתופד מומחה, המתמ...",שדרות חיפה,"שדרות ההסתדרות 55, קניון לב המפרץ, אסותא, חי..."
4,https://www.medonline.co.il/expert/%D7%A7%D7%A...,"ד""ר ופא קסיס",דוקטור,"גריאטריה, פנימית, רפואת כאב, פסיכוגריאטריה","מבוגרים, גיל הזהב",רפואה קונבנציונלית,צפון,"קליניקה: שר""פ פלוס, קניון לב המפרץ, קומה 3, חי...","נווה גנים 12, חיפה",077-2319-148,wafakasis.dr@gmail.com,פורום גריאטריה,"ד""ר ופא קסיס הינה מומחית לרפואה פנימית וגריאטר...",חיפה,"נווה גנים 12, חיפה. wafakasis.dr@gmail.com. ..."
